In [3]:
import pandas as pd

In [4]:
df_sample = pd.read_csv("../data/filtered_transcations10k.csv")

In [5]:
import pandas as pd

# Sample DataFrame (make sure this matches your actual data)
# df_sample = pd.DataFrame({
#     'customer_id': [1, 1, 1, 2, 2, 3, 3, 3],
#     'article_id': [101, 102, 103, 101, 104, 102, 103, 105]
# })

# Initialize the lists
customer_ids = []
article_ids = []
interactions = []

# Iterate through the DataFrame
for index, row in df_sample.iterrows():
    customer_ids.append(row['customer_id'])  # Ensure this matches the actual column name
    article_ids.append(row['article_id'])    # Ensure this matches the actual column name
    interactions.append(1)  # All interactions are 1

# Create the final dictionary
result = {
    'customerid': customer_ids,
    'articleid': article_ids,
    'interaction': interactions  # This should be correct
}

# Create DataFrame from the result
result_df = pd.DataFrame(result)

# Print the result DataFrame
# print(result_df)

# Now, let's create a user-item matrix
user_item_matrix = result_df.pivot_table(index='customerid', columns='articleid', values='interaction', fill_value=0)

# Print the user-item matrix
print(user_item_matrix)


articleid                                           108775015  108775044  \
customerid                                                                 
00022754ec18c5e53757eea8b281632a5c4a499368ecc53...        0.0        0.0   
0006d37aaf7dd84f9bbc02f6cadcb74fd72ebf370bdc5f1...        0.0        0.0   
001e7e640da3c0a2ae3c1444d9fba300646d050e7ba6012...        0.0        0.0   
002975c14a1aecac230ebf40c314fcad04ee3140e9bef25...        0.0        0.0   
002c3a626719d74c4012d90d89f2ac60c7703931f2d3cf2...        0.0        0.0   
...                                                       ...        ...   
ffddfc4b23d005757b91541b6accbe3451205c88dfa5b58...        0.0        0.0   
ffdfd6e1ccacd22ab654e1c94ea5858edd260301073101d...        0.0        0.0   
fff04954c6e484a8deb5ec475e581aefd25d5850d1886f6...        0.0        0.0   
fff8eafb387d2052bffa46ed05c9333e4dd50d86f151127...        0.0        0.0   
ffffaff3905b803d1c7e153a1378a5151e1f34f236ba545...        0.0        0.0   

articleid  

In [6]:
user_item_matrix.to_csv("user_item_matrix.csv")

In [11]:
user_item_matrix.to_pickle("user_item_matrix.pkl")

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate the cosine similarity between users
user_similarity = cosine_similarity(user_item_matrix)

# Convert the similarity matrix into a DataFrame for better readability
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)
print(user_similarity_df)

customerid                                          00022754ec18c5e53757eea8b281632a5c4a499368ecc53a21adfc64a665ba8b  \
customerid                                                                                                             
00022754ec18c5e53757eea8b281632a5c4a499368ecc53...                                                1.0                  
0006d37aaf7dd84f9bbc02f6cadcb74fd72ebf370bdc5f1...                                                0.0                  
001e7e640da3c0a2ae3c1444d9fba300646d050e7ba6012...                                                0.0                  
002975c14a1aecac230ebf40c314fcad04ee3140e9bef25...                                                0.0                  
002c3a626719d74c4012d90d89f2ac60c7703931f2d3cf2...                                                0.0                  
...                                                                                               ...                  
ffddfc4b23d005757b91541b6accbe3451205c88

In [8]:
user_similarity_df.to_csv("user_similarity.csv")

In [12]:
user_similarity_df.to_pickle("user_similarity_df.pkl")

In [9]:
print(list(df_sample['customer_id'])[:10])
print(list(df_sample['customer_id'])[-1])

00401a367c5ac085cb9d4b77c56f3edcabf25153615db982fcc7991d182e10a9
0d6abaf72355724d85f28a0eee0800c9953c5aa89df05f988813de1e83f5033a


In [10]:
def recommend_products_knn(user_id, k=4, n_recommendations=20):
    # Get the k most similar users to the target user
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).index[1:k+1]
    
    # Get items that the similar users interacted with
    similar_users_items = user_item_matrix.loc[similar_users].sum(axis=0)
    
    # Remove items that the target user has already interacted with
    user_items = user_item_matrix.loc[user_id]
    recommendations = similar_users_items[user_items == 0].sort_values(ascending=False).head(n_recommendations)
    
    return recommendations.index.tolist()

# Example: Get recommendations for user with customerid 1
user_id = "0d6abaf72355724d85f28a0eee0800c9953c5aa89df05f988813de1e83f5033a"
recommended_products = recommend_products_knn(user_id)
print(f"Recommended products for user {user_id}: {recommended_products}")


Recommended products for user 0d6abaf72355724d85f28a0eee0800c9953c5aa89df05f988813de1e83f5033a: [690992003, 690992002, 690992001, 690926001, 688658003, 688658001, 688463001, 688432001, 688341001, 687921002, 687921001, 685347003, 685347002, 685347001, 685343001, 684895001, 237347037, 237347036, 237347035, 237347031]
